In [1]:
#!pip install --upgrade google-api-python-client
#!pip install --user google-colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###IMPORTANT-> DO NOT RUN THIS PART UNLESS YOU WANT TO CREATE NEW TRAINING TEST AND DEV DATASETS.

### CREATING TEST, TRAIN AND DEV FROM MEDMENTIONS CORPUS

READING FILES: dev, test, train and corpus

In [ ]:
#read file DEV
devTxt="/content/drive/MyDrive/medNer/corpus_pubtator_pmids_dev.txt"
with open(devTxt, "r") as file1:
    devTxtList = file1.readlines()
#delete \n at the end of every line    
for i in range(0,len(devTxtList)):
  devTxtList[i]=devTxtList[i].split('\n')[0]


#read file TEST
testTxt="/content/drive/MyDrive/medNer/corpus_pubtator_pmids_test.txt"
with open(testTxt, "r") as file2:
    testTxtList = file2.readlines()
#delete \n at the end of every line    
for i in range(0,len(testTxtList)):
  testTxtList[i]=testTxtList[i].split('\n')[0]


#read file TRAIN
trngTxt="/content/drive/MyDrive/medNer/corpus_pubtator_pmids_trng.txt"
with open(trngTxt, "r") as file3:
    trngTxtList = file3.readlines()
#delete \n at the end of every line    
for i in range(0,len(trngTxtList)):
  trngTxtList[i]=trngTxtList[i].split('\n')[0]


#read file CORPUS
corpusTxt="/content/drive/MyDrive/medNer/corpus_pubtator.txt"
with open(corpusTxt, "r") as file4:
    corpusTxtList = file4.readlines()


LOADING TOKENIZER

In [ ]:
#DE PRUEBAS
# http://sentiment.christopherpotts.net/code-data/happyfuntokenizing.py -en bertsio motza
import nltk, re, json
import spacy
from collections import Counter
# Ordena garrantzitsua da  (lehenetik azkenera egiten dire parekatzeak)

# Erabiltzaile-izenak batera mantendu (@ ikurraz hasitako edozein token, eta ondoren A-Z, a-z, 0-9)
regexes=(
# zuriunea ez den beste edozer
r"(?:\w+\.\w+)", 
#symbols
r"(?:[\<\>\=\+|\-|\/|\*\\\'\"|\%|\w]+)",   

# hashtag-ak elkarrekin mantendu (# ikurraz hasitako edozein token, eta ondoren A-Z, a-z, 0-9, _, edo -)
r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)",



)

big_regex="|".join(regexes)

my_extensible_tokenizer = re.compile(big_regex, re.VERBOSE | re.I | re.UNICODE)

def my_extensible_tokenize(text):
    return my_extensible_tokenizer.findall(text)

In [ ]:
import nltk, re, json
import spacy
from collections import Counter

regexes=(
r"(?:T[0-9]+)", 
)

big_regex="|".join(regexes)

semanticTypeTokenizer = re.compile(big_regex, re.VERBOSE | re.I | re.UNICODE)

def semanticTypeTokenizerFunc(text):
    return semanticTypeTokenizer.findall(text)

CREATING A FUNCTION TO NORMALIZE ANY STRING USING THE TOKENIZER

In [ ]:
def normalizeString(string):
  #print(string)
  last_char = string[-1]
  isEndOrComma=None

  if last_char == '.':
    isEndOrComma="END"
  if last_char == ',':
    isEndOrComma="COMMA"
    
  string=my_extensible_tokenize(string)
  #print(string)
  if len(string)==0:
    return "", isEndOrComma
  else:
    return string[0], isEndOrComma

In [ ]:
normalizeString("(hola,)")

('hola', None)

In [ ]:
def normalizeSemanticType(string):   
  lista=semanticTypeTokenizerFunc(string) 
  if len(lista)==0:
    return ""
  return lista[0]

GETTING ORDER MOST APPEARING OF SEMANTIC TYPES

In [ ]:
from collections import Counter
import collections
import operator
inPaper=False
titleRow=0
paperRow=1
SematicTypeAppearence=[]

#COMMENT NEXT TWO LINES FOR WHOLE CORPUS
#maxAmountOfPapers=100
paperKont=0

with open('/content/drive/MyDrive/medNer/medMentions-train-SemType-version.tsv', 'r') as read_file:

  for i in range(0,len(corpusTxtList)):
    
    #beginning of paper
    if(inPaper==False):
      #number of paper
      paperNumber=corpusTxtList[i+2].split('\t')[0]
      #title
      currentTitle=corpusTxtList[i]
      #paper
      currentPaper=corpusTxtList[i+1]
      #restarting BIbek
      

    #mid paper
    if titleRow!=i and paperRow!=i and corpusTxtList[i]!='\n':
      #vector for storing begin and inside words
      #print(corpusTxtList[i].split('\t')[-1])
      SematicTypeAppearence.append(normalizeSemanticType(corpusTxtList[i].split('\t')[4]))

    inPaper=True
    #end of paper
    if corpusTxtList[i]=='\n':
      titleRow=i+1
      paperRow=i+2
      inPaper=False
      if paperNumber in trngTxtList:
        paperKont+=1

semanticDictionary=Counter(SematicTypeAppearence)
#collections.OrderedDict(sorted(semanticDictionary.values()))
dictionary=sorted(semanticDictionary.items(), key=operator.itemgetter(1))
dictionary

[('T021', 2),
 ('T127', 2),
 ('T095', 19),
 ('T010', 20),
 ('T203', 20),
 ('T194', 25),
 ('T011', 27),
 ('T094', 36),
 ('T192', 52),
 ('T125', 54),
 ('T014', 58),
 ('T200', 61),
 ('T085', 61),
 ('T072', 79),
 ('T171', 99),
 ('T089', 102),
 ('T071', 109),
 ('T017', 134),
 ('T102', 154),
 ('T195', 180),
 ('T064', 188),
 ('T020', 189),
 ('T069', 190),
 ('T018', 267),
 ('T190', 271),
 ('T013', 276),
 ('T066', 279),
 ('T050', 288),
 ('T103', 289),
 ('T051', 292),
 ('T068', 295),
 ('T092', 300),
 ('T120', 311),
 ('T019', 347),
 ('T075', 365),
 ('T087', 365),
 ('T129', 365),
 ('T197', 368),
 ('T008', 429),
 ('T053', 447),
 ('T012', 465),
 ('T022', 517),
 ('T196', 525),
 ('T065', 554),
 ('T090', 565),
 ('T130', 579),
 ('T131', 581),
 ('T049', 594),
 ('T030', 602),
 ('T093', 604),
 ('T004', 612),
 ('T086', 624),
 ('T016', 732),
 ('T104', 799),
 ('T099', 816),
 ('T056', 863),
 ('T034', 874),
 ('T057', 891),
 ('T063', 923),
 ('T091', 938),
 ('T038', 946),
 ('T054', 982),
 ('T055', 1012),
 ('T001'

In [ ]:
n_items=[]
n_number_of_ner_tags=15

for i in reversed(range(len(dictionary)-n_number_of_ner_tags, len(dictionary))):
  n_items.append(dictionary[i][0])

n_items

['T080',
 'T169',
 'T081',
 'T033',
 'T116',
 'T047',
 'T061',
 'T109',
 'T079',
 'T078',
 'T170',
 'T062',
 'T082',
 'T052',
 'T098']

CREATING FUNCTIONS FOR CREATING TSV FILES FOR TRAINING DEV AND TEST

In [ ]:
import csv

def textToTokens(title, paper):
  title=title.split()
  paper=paper.split()
  title[0]=title[0].split('|t|')[1]
  paper[0]=paper[0].split('|a|')[1]
  text = title+paper
  #print(text)
  return text
  
def textAndBItoTSV(tsv_writer, text, BI):#

  textCount=0
  for i in BI:
    word = i[0].split()
    SEMTYPE=i[1].rstrip("\n")
    #print(word[0])
    #print(text[textCount])
    for w in range(0,len(word)):
      #print("for")
      if textCount<len(text): #puede que sea <=

        #normalize strings
        #print("word[w]"+str(word[w]))
        word[w], isEndOrComma=normalizeString(word[w])
        #print("text[textCount"+str(textCount)+"]"+str(text[textCount]))
        text[textCount], isEndOrComma=normalizeString(text[textCount])

        if word[w]==text[textCount]:
          if w==0:
            #print(word[w])
            #print(text[textCount])
            if word[w]!="" and text[textCount]!="":
              if SEMTYPE in n_items:
                tsv_writer.writerow([text[textCount].lower(), 'B-'+SEMTYPE ])
              else:
                tsv_writer.writerow([text[textCount].lower(), 'O' ])
              #print("B")
              #print("word: "+str(word[w]))
              #print("textCount: "+str(text[textCount]))
              if isEndOrComma=="END":
                tsv_writer.writerow([".", 'O'])
                tsv_writer.writerow(["", ''])
              if isEndOrComma=="COMMA":
                tsv_writer.writerow([",", 'O'])

            textCount+=1
            
          else:
            if word[w]!="" and text[textCount]!="":
              if SEMTYPE in n_items:
                tsv_writer.writerow([text[textCount].lower(), 'I-'+SEMTYPE ])
              else:
                tsv_writer.writerow([text[textCount].lower(), 'O' ])
              #print("I")
              #print("word: "+str(word[w]))
              #print("textCount: "+str(text[textCount]))
              if isEndOrComma=="END":
                tsv_writer.writerow([".", 'O'])
                tsv_writer.writerow(["", ''])
              if isEndOrComma=="COMMA":
                tsv_writer.writerow([",", 'O'])
            textCount+=1
            
        else:
          justEntered=True
          while textCount<len(text)-1:
            #print("while")
            if justEntered==False:
              #print("word[w]"+str(word[w]))
              word[w], isEndOrComma=normalizeString(word[w])
              #print(normalizeString(text[textCount]))
              #print("text[textCount"+str(textCount)+"]"+str(text[textCount]))
              text[textCount], isEndOrComma=normalizeString(text[textCount])
            justEntered=False
            if word[w]!=text[textCount]:
              if word[w]!="" and text[textCount]!="":
                tsv_writer.writerow([text[textCount].lower(), 'O'])
                if isEndOrComma=="END":
                  tsv_writer.writerow([".", 'O'])
                  tsv_writer.writerow(["", ''])
                if isEndOrComma=="COMMA":
                  tsv_writer.writerow([",", 'O'])

              #print("O")
              #print("word: "+str(word[w]))
              #print("textCount: "+str(text[textCount]))
              textCount+=1
            else:
              if w==0:
                #print("B")
                #print("word: "+str(word[w]))
                #print("textCount: "+str(text[textCount]))
                if word[w]!="" and text[textCount]!="":
                  if SEMTYPE in n_items:
                    tsv_writer.writerow([text[textCount].lower(), 'B-'+SEMTYPE ])
                  else:
                    tsv_writer.writerow([text[textCount].lower(), 'O' ])
                  if isEndOrComma=="END":
                    tsv_writer.writerow([".", 'O'])
                    tsv_writer.writerow(["", ''])
                  if isEndOrComma=="COMMA":
                    tsv_writer.writerow([",", 'O'])
                textCount+=1
                break
              else:
                #print("I")
                #print("word: "+str(word[w]))
                #print("textCount: "+str(text[textCount]))
                if word[w]!="" and text[textCount]!="":
                  if SEMTYPE in n_items:
                    tsv_writer.writerow([text[textCount].lower(), 'I-'+SEMTYPE ])
                  else:
                    tsv_writer.writerow([text[textCount].lower(), 'O' ])
                  if isEndOrComma=="END":
                    tsv_writer.writerow([".", 'O'])
                    tsv_writer.writerow(["", ''])
                  if isEndOrComma=="COMMA":
                    tsv_writer.writerow([",", 'O'])
                textCount+=1
                break
         
        



CREATING TRAINING SET

In [ ]:
inPaper=False
titleRow=0
paperRow=1
BIbek=[]

#COMMENT NEXT TWO LINES FOR WHOLE CORPUS
#maxAmountOfPapers=100
paperKont=0

with open('/content/drive/MyDrive/medNer/medMentions-train-SemType15-version.tsv', 'wt') as out_file:
  tsv_writer = csv.writer(out_file, delimiter='\t')

  for i in range(0,len(corpusTxtList)):
    
    #beginning of paper
    if(inPaper==False):
      #number of paper
      paperNumber=corpusTxtList[i+2].split('\t')[0]
      #title
      currentTitle=corpusTxtList[i]
      #paper
      currentPaper=corpusTxtList[i+1]
      #restarting BIbek
      

    #mid paper
    if titleRow!=i and paperRow!=i and corpusTxtList[i]!='\n':
      #vector for storing begin and inside words
      #print(corpusTxtList[i].split('\t')[-1])
      elemOfBek=[corpusTxtList[i].split('\t')[3],normalizeSemanticType(corpusTxtList[i].split('\t')[4])]
      BIbek.append(elemOfBek)
    inPaper=True
    #end of paper
    if corpusTxtList[i]=='\n':
      titleRow=i+1
      paperRow=i+2
      inPaper=False
      if paperNumber in trngTxtList:
        #COMMENT NEXT LINE FOR WHOLE CORPUS
        #if paperKont<maxAmountOfPapers:
        textAndBItoTSV(tsv_writer, textToTokens(currentTitle, currentPaper),BIbek)
          #COMMENT NEXT LINE FOR WHOLE CORPUS
          #paperKont+=1
      BIbek=[]


CREATING TEST SET

In [ ]:
inPaper=False
titleRow=0
paperRow=1
BIbek=[]

#COMMENT NEXT TWO LINES FOR WHOLE CORPUS
#maxAmountOfPapers=40
paperKont=0

with open('/content/drive/MyDrive/medNer/medMentions-test-SemType15-version.tsv', 'wt') as out_file:
  tsv_writer = csv.writer(out_file, delimiter='\t')

  for i in range(0,len(corpusTxtList)):
    
    #beginning of paper
    if(inPaper==False):
      #number of paper
      paperNumber=corpusTxtList[i+2].split('\t')[0]
      #title
      currentTitle=corpusTxtList[i]
      #paper
      currentPaper=corpusTxtList[i+1]
      #restarting BIbek
      

    #mid paper
    if titleRow!=i and paperRow!=i and corpusTxtList[i]!='\n':
      #vector for storing begin and inside words
      elemOfBek=[corpusTxtList[i].split('\t')[3],normalizeSemanticType(corpusTxtList[i].split('\t')[4])]
      BIbek.append(elemOfBek)
    inPaper=True
    #end of paper
    if corpusTxtList[i]=='\n':
      titleRow=i+1
      paperRow=i+2
      inPaper=False
      if paperNumber in testTxtList:
        #COMMENT NEXT LINE FOR WHOLE CORPUS
        #if paperKont<maxAmountOfPapers:
        textAndBItoTSV(tsv_writer, textToTokens(currentTitle, currentPaper),BIbek)
          #COMMENT NEXT LINE FOR WHOLE CORPUS
          #paperKont+=1
      BIbek=[]


CREATING DEV SET

In [ ]:
inPaper=False
titleRow=0
paperRow=1
BIbek=[]

#COMMENT NEXT TWO LINES FOR WHOLE CORPUS
#maxAmountOfPapers=40
paperKont=0

with open('/content/drive/MyDrive/medNer/medMentions-dev-SemType15-version.tsv', 'wt') as out_file:
  tsv_writer = csv.writer(out_file, delimiter='\t')

  for i in range(0,len(corpusTxtList)):
    
    #beginning of paper
    if(inPaper==False):
      #number of paper
      paperNumber=corpusTxtList[i+2].split('\t')[0]
      #title
      currentTitle=corpusTxtList[i]
      #paper
      currentPaper=corpusTxtList[i+1]
      #restarting BIbek
      

    #mid paper
    if titleRow!=i and paperRow!=i and corpusTxtList[i]!='\n':
      #vector for storing begin and inside words
      elemOfBek=[corpusTxtList[i].split('\t')[3],normalizeSemanticType(corpusTxtList[i].split('\t')[4])]
      BIbek.append(elemOfBek)
    inPaper=True
    #end of paper
    if corpusTxtList[i]=='\n':
      titleRow=i+1
      paperRow=i+2
      inPaper=False
      if paperNumber in devTxtList:
        #COMMENT NEXT LINE FOR WHOLE CORPUS
        #if paperKont<maxAmountOfPapers:
          textAndBItoTSV(tsv_writer, textToTokens(currentTitle, currentPaper),BIbek)
          #COMMENT NEXT LINE FOR WHOLE CORPUS
          #paperKont+=1

      BIbek=[]


###YOU CAN TEST THE TRAINING PART IF YOU ARE WILLING TO TRAIN AN ALREADY EXISTING MODEL

###TRAINING

Flair 

In [3]:
pip install --upgrade flair

     |████████████████████████████████| 286kB 5.0MB/s 
     |████████████████████████████████| 1.2MB 8.8MB/s 
     |████████████████████████████████| 798kB 18.0MB/s 
     |████████████████████████████████| 19.7MB 9.9MB/s 
     |████████████████████████████████| 71kB 8.7MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 983kB 35.5MB/s 
     |████████████████████████████████| 776.8MB 23kB/s 
     |████████████████████████████████| 2.1MB 27.0MB/s 
     |████████████████████████████████| 61kB 7.8MB/s 
     |████████████████████████████████| 3.3MB 21.4MB/s 
     |████████████████████████████████| 901kB 25.9MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-cp37-none-any.whl size=9693 sha256=03c693020b812a8fa0733bdb1f1c5f43d3d8124ac05782d306ed6957d81e38d4
  Stored in directory: /root/.cache/pip/wheels/81/d0/d7/d9983facc6f2775411803e0e2d30ebf98efbf2fc6e57701e09
Successf

In [ ]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus, NCBI_DISEASE
from flair.embeddings import WordEmbeddings, StackedEmbeddings, FlairEmbeddings#, TokenEmbeddings
from typing import List

In [ ]:
# define columns
columns = {0: 'text', 1: 'ner'}

# this is the folder in which train, test and dev files reside
data_folder = '/content/drive/MyDrive/medNer'

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='medMentions-train-SemType10-version.tsv',
                              test_file='medMentions-test-SemType10-version.tsv',
                              dev_file='medMentions-dev-SemType10-version.tsv')

# TODO obtain and print corpus statistics (output below obtained with the BASQUE NER corpus)
print(corpus.obtain_statistics())

2021-05-11 08:44:26,801 Reading data from /content/drive/MyDrive/medNer
2021-05-11 08:44:26,809 Train: /content/drive/MyDrive/medNer/medMentions-train-SemType10-version.tsv
2021-05-11 08:44:26,812 Dev: /content/drive/MyDrive/medNer/medMentions-dev-SemType10-version.tsv
2021-05-11 08:44:26,816 Test: /content/drive/MyDrive/medNer/medMentions-test-SemType10-version.tsv
{
    "TRAIN": {
        "dataset": "TRAIN",
        "total_number_of_documents": 25094,
        "number_of_documents_per_class": {},
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 654178,
            "min": 2,
            "max": 173,
            "avg": 26.069100183310752
        }
    },
    "TEST": {
        "dataset": "TEST",
        "total_number_of_documents": 8312,
        "number_of_documents_per_class": {},
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 219149,
            "min": 2,
            "max": 142,
            "avg": 26.3

In [ ]:
# 2. what tag do we want to predict?
tag_type = 'ner'

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
print(tag_dictionary.idx2item)

[b'<unk>', b'O', b'B-T116', b'B-T047', b'I-T047', b'B-T079', b'B-T169', b'I-T169', b'B-T033', b'I-T033', b'B-T081', b'I-T116', b'B-T109', b'B-T080', b'I-T109', b'B-T078', b'B-T061', b'I-T061', b'I-T080', b'I-T081', b'I-T078', b'I-T079', b'<START>', b'<STOP>']


You can choose the embeddings. The ideal is the following combination:FlairEmbeddings('pubmed-forward'), FlairEmbeddings('pubmed-backward'), WordEmbeddings("pubmed"). But due to memory issues just WordEmbeddings("pubmed") is the best choice.

In [ ]:
embeddings : StackedEmbeddings = StackedEmbeddings([
                                        #WordEmbeddings('glove'),
                                        #FlairEmbeddings('news-forward'),
                                        #FlairEmbeddings('news-backward')
                                        #FlairEmbeddings('pubmed-forward')
                                        #FlairEmbeddings('pubmed-backward')
                                        WordEmbeddings("pubmed"),

                                        # flair embeddings trained on PubMed and PMC
                                        #FlairEmbeddings("pubmed-forward"),
                                        #FlairEmbeddings("pubmed-backward"),
                                       ])

2021-05-11 08:44:49,403 https://flair.informatik.hu-berlin.de/resources/embeddings/token/pubmed_pmc_wiki_sg_1M.gensim.vectors.npy not found in cache, downloading to /tmp/tmpq2cdzrtn


100%|██████████| 800000128/800000128 [00:56<00:00, 14231511.82B/s]

2021-05-11 08:45:46,052 copying /tmp/tmpq2cdzrtn to cache at /root/.flair/embeddings/pubmed_pmc_wiki_sg_1M.gensim.vectors.npy


2021-05-11 08:45:48,947 removing temp file /tmp/tmpq2cdzrtn
2021-05-11 08:45:49,622 https://flair.informatik.hu-berlin.de/resources/embeddings/token/pubmed_pmc_wiki_sg_1M.gensim not found in cache, downloading to /tmp/tmpymy3_jwj


100%|██████████| 53979687/53979687 [00:02<00:00, 18236691.92B/s]

2021-05-11 08:45:52,971 copying /tmp/tmpymy3_jwj to cache at /root/.flair/embeddings/pubmed_pmc_wiki_sg_1M.gensim


2021-05-11 08:45:53,033 removing temp file /tmp/tmpymy3_jwj


In [ ]:
#from gensim.models import KeyedVectors
#word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)
#x = word2vec.word_vec("test")
#EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz'

In [ ]:
from flair.models import SequenceTagger

tagger : SequenceTagger = SequenceTagger(hidden_size=256,
                                       embeddings=embeddings,
                                       tag_dictionary=tag_dictionary,
                                       tag_type=tag_type,
                                       use_crf=True)
print(tagger)


SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('pubmed')
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=200, out_features=200, bias=True)
  (rnn): LSTM(200, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=24, bias=True)
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)


In [ ]:
from flair.trainers import ModelTrainer
trainer : ModelTrainer = ModelTrainer(tagger, corpus)
trainer.train('/content/drive/MyDrive/medNer',
              learning_rate=0.1,
              mini_batch_size=16,
              max_epochs=30)

2021-05-11 08:45:57,634 ----------------------------------------------------------------------------------------------------
2021-05-11 08:45:57,636 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('pubmed')
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=200, out_features=200, bias=True)
  (rnn): LSTM(200, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=24, bias=True)
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)"
2021-05-11 08:45:57,638 ----------------------------------------------------------------------------------------------------
2021-05-11 08:45:57,640 Corpus: "Corpus: 25094 train + 8284 dev + 8312 test sentences"
2021-05-11 08:45:57,644 ----------------------------------------------------------------------------------------------------
2021-05-11 08:45:57,647 Parameters:
2021-05-11 08:45:57,649  - learn

LOAD MODEL

In [4]:
from flair.data import Sentence
from flair.models import SequenceTagger
# load the trained model
model = SequenceTagger.load('/content/drive/MyDrive/medNer/best-model.pt')


2021-05-12 11:47:03,350 loading file /content/drive/MyDrive/medNer/best-model.pt


RESULTS

In [5]:
from segtok.segmenter import split_single
#read file
Text="/content/drive/MyDrive/medNer/paper.txt"
with open(Text, "r") as file1:
    FileasList = file1.readlines()
TextStr= FileasList[0]

#NER
#2.Whole text predict
TextStrAll=Sentence(TextStr)
model.predict(TextStrAll)
print(TextStrAll.to_tagged_string())

The purpose of this study was to evaluate the effect <B-T080> of femoral tunnel orientation , drilled through the accessory anteromedial ( AAM ) portal or the high AM portal in anatomic anterior cruciate ligament ( ACL ) reconstruction <B-T061> . In 16 cadaver knees , using o'clock method , centers of the ACL femoral footprint were drilled with an 8-mm reamer via an AAM portal ( eight knees ) or a high AM portal ( eight knees ) . Computed tomography ( CT ) scans were taken of each knee . Three-dimensional ( 3D ) models were constructed to identify the femoral tunnel orientation and to create femoral tunnel virtual cylinders for measuring tunnel angles and length <B-T081> . In two of the 16 specimens , we observed a posterior femoral cortex blowout ( PFCB ) when drilling through a high AM portal . When drilled through the high AM portal , the femoral tunnel length <B-T081> was significantly shorter than when using an AAM portal ( 30.3 ± 3.8 mm and 38.2 ± 3.1 mm , p < 0.001 ) . The femor

The real paper:

In [ ]:
28366539|t|Influence of the different anteromedial portal on femoral tunnel orientation during anatomic ACL reconstruction
28366539|a|The purpose of this study was to evaluate the effect of femoral tunnel orientation, drilled through the accessory anteromedial (AAM) portal or the high AM portal in anatomic anterior cruciate ligament (ACL) reconstruction. In 16 cadaver knees, using o'clock method, centers of the ACL femoral footprint were drilled with an 8-mm reamer via an AAM portal (eight knees) or a high AM portal (eight knees). Computed tomography (CT) scans were taken of each knee. Three-dimensional (3D) models were constructed to identify the femoral tunnel orientation and to create femoral tunnel virtual cylinders for measuring tunnel angles and length. In two of the 16 specimens, we observed a posterior femoral cortex blowout (PFCB) when drilling through a high AM portal. When drilled through the high AM portal, the femoral tunnel length was significantly shorter than when using an AAM portal (30.3 ± 3.8 mm and 38.2 ± 3.1 mm, p < 0.001). The femoral tunnel length was significantly shorter in the group with PFCB compared to the group with no PFCB (25.9 ± 0.6 mm and 35.5 ± 4.5 mm, p = 0.011). The axial obliquity of the high AM portal was significantly higher than that of the AAM portal (52.2 ± 5.9° and 43.0 ± 2.3°, p = 0.003). In anatomic ACL reconstruction, a mal-positioned AM portal can cause abnormal tunnel orientation, which may lead to mechanical failure during ACL reconstruction. Therefore, it is important to select accurate AM portal positioning, and possibly using an AAM portal by measuring an accurate position when drilling a femoral tunnel in anatomic ACL reconstruction.
28366539	17	26	different	T080	C1705242
28366539	27	46	anteromedial portal	T061	C0185154
28366539	50	64	femoral tunnel	T023	C0015811
28366539	65	76	orientation	T082	C1704322
28366539	84	92	anatomic	T080	C0220784
28366539	93	111	ACL reconstruction	T061	C0188185
28366539	145	153	evaluate	T058	C0220825
28366539	158	164	effect	T080	C1280500
28366539	168	182	femoral tunnel	T023	C0015811
28366539	183	194	orientation	T082	C1704322
28366539	196	203	drilled	T061	C0337279
28366539	216	251	accessory anteromedial (AAM) portal	T061	C0185154
28366539	259	273	high AM portal	T061	C0185154
28366539	277	333	anatomic anterior cruciate ligament (ACL) reconstruction	T061	C0188185
28366539	341	348	cadaver	T017	C0006629
28366539	349	354	knees	T023	C0022742
28366539	362	376	o'clock method	T061	C1293156
28366539	378	385	centers	T082	C0205099
28366539	393	414	ACL femoral footprint	T023	C0078960
28366539	420	427	drilled	T061	C0337279
28366539	441	447	reamer	T074	C3853551
28366539	455	465	AAM portal	T061	C0185154
28366539	473	478	knees	T023	C0022742
28366539	485	499	high AM portal	T061	C0087111
28366539	507	512	knees	T023	C0022742
28366539	515	545	Computed tomography (CT) scans	T060	C0040405
28366539	565	569	knee	T023	C0022742
28366539	571	600	Three-dimensional (3D) models	T075	C0026336
28366539	634	648	femoral tunnel	T023	C0015811
28366539	649	660	orientation	T082	C1704322
28366539	668	674	create	T052	C1706214
28366539	675	707	femoral tunnel virtual cylinders	T023	C0015811
28366539	712	721	measuring	T080	C0444706
28366539	722	735	tunnel angles	T030	C0229984
28366539	740	746	length	T081	C1444754
28366539	765	774	specimens	T167	C0370003
28366539	779	787	observed	T169	C1441672
28366539	790	822	posterior femoral cortex blowout	T046	C0021890
28366539	824	828	PFCB	T046	C0021890
28366539	835	843	drilling	T061	C0337279
28366539	844	851	through	T169	C0332273
28366539	854	868	high AM portal	T061	C0185154
28366539	875	882	drilled	T061	C0337279
28366539	883	890	through	T169	C0332273
28366539	895	909	high AM portal	T061	C0185154
28366539	915	929	femoral tunnel	T023	C0015811
28366539	930	936	length	T081	C1444754
28366539	941	954	significantly	T078	C0750502
28366539	955	962	shorter	T081	C1806781
28366539	982	992	AAM portal	T061	C0185154
28366539	1043	1057	femoral tunnel	T023	C0015811
28366539	1058	1064	length	T081	C1444754
28366539	1069	1082	significantly	T078	C0750502
28366539	1083	1090	shorter	T081	C1806781
28366539	1098	1103	group	T078	C0441833
28366539	1109	1113	PFCB	T046	C0021890
28366539	1114	1122	compared	T052	C1707455
28366539	1130	1135	group	T078	C0441833
28366539	1141	1148	no PFCB	T033	C0243095
28366539	1199	1214	axial obliquity	T082	C0205315
28366539	1222	1236	high AM portal	T061	C0185154
28366539	1241	1254	significantly	T078	C0750502
28366539	1255	1261	higher	T080	C0205250
28366539	1279	1289	AAM portal	T061	C0185154
28366539	1335	1343	anatomic	T080	C0220784
28366539	1344	1362	ACL reconstruction	T061	C0188185
28366539	1366	1380	mal-positioned	T082	C0333042
28366539	1381	1390	AM portal	T061	C0185154
28366539	1401	1409	abnormal	T033	C0205161
28366539	1410	1416	tunnel	T023	C0015811
28366539	1417	1428	orientation	T082	C1704322
28366539	1448	1458	mechanical	T169	C0443254
28366539	1474	1492	ACL reconstruction	T061	C0188185
28366539	1531	1539	accurate	T080	C0443131
28366539	1540	1549	AM portal	T061	C0185154
28366539	1550	1561	positioning	T082	C0733755
28366539	1585	1595	AAM portal	T061	C0185154
28366539	1599	1608	measuring	T080	C0444706
28366539	1612	1620	accurate	T080	C0443131
28366539	1621	1629	position	T082	C0733755
28366539	1635	1643	drilling	T061	C0337279
28366539	1646	1660	femoral tunnel	T023	C0015811
28366539	1664	1672	anatomic	T080	C0220784
28366539	1673	1691	ACL reconstruction	T061	C0188185